# RunnableWithMessageHistory
- 사용자와의 대화 history를 prompt에 넣어줌으로써 llm이 이전 대화내용을 계속 기억하면서 대화를 이어나갈 수 있도록 하는 클래스
- In-Memory, Redis 등 다양한 저장소와 활용할 수 있다

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="gemma:7b")
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {ability}에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
chain = prompt | model

# In-Memory

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = (
    RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input", # 사용자 입력 메세지 key
        history_messages_key="history", # history 메세지 key
    )
)

In [5]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 125d7716-72f7-4179-9095-cbae4e914631 not found for run ee89c20f-cc96-4578-949a-8d88e4c05be7. Treating as a root run.


AIMessage(content='cosine는 삼각형의 변의 길이와 빗변의 길이의 비율입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T15:50:18.315510348Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6431273104, 'load_duration': 6055012315, 'prompt_eval_count': 57, 'prompt_eval_duration': 107238000, 'eval_count': 23, 'eval_duration': 220816000}, id='run-f370d42d-d661-4c6b-960c-9136f2ca2fbe-0')

In [6]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 영어로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 411a5cdc-a423-4ee3-9826-b632e6a62012 not found for run a469105e-b9d0-4992-a6e4-779f691cdc08. Treating as a root run.


AIMessage(content='Cosine is the ratio of the length of the adjacent side to the length of the hypotenuse in a triangle.</end_of_turn>', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T15:50:28.248067562Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 542538623, 'load_duration': 2558808, 'prompt_eval_count': 62, 'prompt_eval_duration': 104464000, 'eval_count': 31, 'eval_duration': 301216000}, id='run-8b616b4d-4bcc-4afc-a8bb-7c01b4c6e783-0')

In [7]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)

Parent run 9464199f-9574-46c6-9377-11791dc95579 not found for run efad7424-4869-488f-8c84-7bdd4bab69d6. Treating as a root run.


AIMessage(content='이전의 내용을 제공해주시면 해당 내용에 대한 응답을 드립니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T15:50:43.971820397Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 455965292, 'load_duration': 2227307, 'prompt_eval_count': 31, 'prompt_eval_duration': 102865000, 'eval_count': 23, 'eval_duration': 220949000}, id='run-f0d71c27-d3c1-4ebf-ab30-c1d90d68303e-0')

history_factory_config 설정을 통해 session_id 대신에 key 값을 custom 할 수 있다

In [9]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[  # 기존의 "session_id" 설정을 대체하게 됩니다.
        ConfigurableFieldSpec(
            id="user_id",  # get_session_history 함수의 첫 번째 인자로 사용됩니다.
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # get_session_history 함수의 두 번째 인자로 사용됩니다.
            annotation=str,
            name="Conversation ID",
            description="대화의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
    ],
)

In [10]:
with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "what is cosine?"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

Parent run ba574798-1be0-4402-bcac-5f7ab4e1ea0d not found for run 711ddc5a-c8da-4092-b815-a26e44a489a1. Treating as a root run.


AIMessage(content='코사인은 삼각형의 빗변과 밑변의 길이의 비율입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T15:53:15.575346917Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 458904410, 'load_duration': 2396898, 'prompt_eval_count': 20, 'prompt_eval_duration': 102418000, 'eval_count': 23, 'eval_duration': 220688000}, id='run-447f44a2-bd41-40f5-9396-269363ba4b1e-0')

In [11]:
with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "이전의 답변을 영어로 작성해 주세요"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

Parent run 6a61156d-d3cd-42e2-a5e3-f4e09c86c163 not found for run 40a77660-f934-4ce1-b565-51755c771b9f. Treating as a root run.


AIMessage(content='Cosine is the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle.</end_turn>', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T15:53:27.96106814Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 530860050, 'load_duration': 2618459, 'prompt_eval_count': 48, 'prompt_eval_duration': 103583000, 'eval_count': 30, 'eval_duration': 290940000}, id='run-a701b5b2-ca48-4734-91ad-129c9533546b-0')

`input_messages_key`를 생략하면 dict 대신 메세지 객체를 넣어줘야 한다.

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOllama(model="gemma:7b")})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]


# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 332d7a6e-a956-4cd0-992f-13e75f7bd3ea not found for run 59db3202-ddff-4c27-86b0-f896872741a4. Treating as a root run.


{'output_message': AIMessage(content='**Cosine** is a mathematical function that relates the lengths of the adjacent and hypotenuse sides of a right triangle. It is typically denoted as **cos(θ)**, where θ is the angle opposite the adjacent side.\n\n**Definition:**\n\n$$\\cos(θ) = \\frac{\\text{adjacent side}}{\\text{hypotenuse}}$$\n\n**Geometric Interpretation:**\n\n- The cosine of an angle is the ratio of the length of the adjacent side (the side next to the angle) to the length of the hypotenuse (the longest side in a right triangle).\n- The cosine function is positive for acute angles (angles less than 90°) and negative for obtuse angles (angles greater than 90°).\n\n**Mathematical Representation:**\n\n$$ \\cos(θ) = \\sum_{n=0}^\\infty (-1)^n \\frac{θ^n}{n!} $$\n\n**Properties:**\n\n- The cosine of 0° is 1.\n- The cosine of 90° is 0.\n- The cosine function is periodic with a period of 360°.\n- The cosine of a complementary angle is equal to the cosine of the original angle.\n\n**Ap

`output_messages_key`를 생략하면 dict가 아닌 AIMessage 객체를 반환한다

In [6]:
with_message_history = RunnableWithMessageHistory(
    ChatOllama(model="gemma:7b"),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)

In [7]:
with_message_history.invoke(
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    config={"configurable": {"session_id": "abc"}}
)

Parent run e34d2b3b-0ec1-4abe-9edd-09461815cd85 not found for run 776b560b-387b-4b88-a307-520e630b3447. Treating as a root run.


AIMessage(content='코사인은 삼각형의 빗변과 밑변 사이의 관계를 나타내는 수학 함수입니다. \n\n$$ \\text{cosa}(x) = \\frac{\\text{adj}}{\\text{hyp}}$$\n\n여기서:\n\n* adj는 삼각형의 빗변의 길이\n* hyp는 삼각형의 높이 \n\n코사인 값은 -1부터 1까지의 범위를 가지며, 삼각형의 빗변과 밑변의 길이에 따라 값이 달라집니다. 특히, \n\n* **cosa(0) = 1** (직각삼각형의 경우)\n* **cosa(90°) = 0** (직각 삼각형의 경우)\n* **cosa(180°) = -1** (직각 삼각형의 경우)\n\n코사인 함수는 삼각형의 넓이를 계산하고, 벡터 분석 및 기하학 문제를 해결하는 데 사용됩니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T16:03:58.00424964Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2575289724, 'load_duration': 2286738, 'prompt_eval_count': 45, 'prompt_eval_duration': 103752000, 'eval_count': 224, 'eval_duration': 2334723000}, id='run-776b560b-387b-4b88-a307-520e630b3447-0')

In [8]:
from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOllama(model="gemma:7b"),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)

In [10]:
with_message_history.invoke({"input_messages": "코사인의 의미는 무엇인가요?"}, config={"configurable": {"session_id": "abc"}})

Parent run 2a4e899b-66a6-417f-877d-3c1ff5dd1def not found for run 895df607-cf71-44dc-a8d8-f27f86f0750a. Treating as a root run.


AIMessage(content='코사인은 삼각형의 빗변과 밑변 사이의 관계를 나타내는 수학 함수입니다. \n\n$$ \\text{cosa}(x) = \\frac{\\text{adj}}{\\text{hyp}}$$\n\n여기서:\n\n* adj는 삼각형의 빗변의 길이\n* hyp는 삼각형의 높이 \n\n코사인 값은 -1부터 1까지의 범위를 가지며, 삼각형의 빗변과 밑변의 길이에 따라 값이 달라집니다. 코사인 함수는 삼각형의 넓이를 계산하고, 벡터 분석 및 기하학 문제를 해결하는 데 사용됩니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T16:05:10.844703809Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1976385771, 'load_duration': 2611839, 'prompt_eval_count': 45, 'prompt_eval_duration': 105370000, 'eval_count': 156, 'eval_duration': 1643100000}, id='run-c67bbc98-89d3-4bbf-a2f9-ad6ed9e47c5c-0')

# Redis 영구 저장소
- `RedisChatMessageHistory` 객체를 사용하여 redis에 대화 history를 받아오고 저장한다

In [12]:
REDIS_URL = "redis://localhost:6379"

In [13]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [16]:
from langchain_community.chat_message_histories import RedisChatMessageHistory

model = ChatOllama(model="gemma:7b")
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {ability}에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])
chain = prompt | model

def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 세션 ID를 기반으로 RedisChatMessageHistory 객체를 반환합니다.
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    chain,  # 실행 가능한 객체
    get_message_history,  # 메시지 기록을 가져오는 함수
    input_messages_key="input",  # 입력 메시지의 키
    history_messages_key="history",  # 기록 메시지의 키
)

In [17]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 옵션으로 세션 ID를 "redis123" 로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)

Parent run 004f7255-07b7-4981-a128-a7c506e5ffc9 not found for run 582deab8-4c3d-43d4-be7a-f8b4189d8646. Treating as a root run.


AIMessage(content='cosine는 삼각형의 빗변과 밑변 사이의 관계를 나타내는 수학 함수입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T16:08:27.875719398Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 496806136, 'load_duration': 2631913, 'prompt_eval_count': 47, 'prompt_eval_duration': 104465000, 'eval_count': 26, 'eval_duration': 259017000}, id='run-ce2f9fd0-fe3f-4d82-b2be-8b7181a41ab1-0')

In [18]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 영어로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "foobar"로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)

Parent run 02d0d8d1-73a0-495d-9705-733020b559dd not found for run 5df402e8-e783-4aa2-bb91-d8b2e04a0ad9. Treating as a root run.


AIMessage(content='Cosine is a mathematical function that represents the ratio of the adjacent side to the hypotenuse in a triangle.</end_of_turn>', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T16:08:44.672938772Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 548186021, 'load_duration': 2998858, 'prompt_eval_count': 49, 'prompt_eval_duration': 104691000, 'eval_count': 30, 'eval_duration': 301299000}, id='run-f263ac4e-ac7f-4c7e-827f-975e12887c57-0')

In [19]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "redis456"로 지정합니다.
    config={"configurable": {"session_id": "redis456"}},
)

Parent run a1a34c94-eb38-4c8a-882e-91b96b6ce890 not found for run ffb0105f-b5f1-43f6-b296-947200ca55e1. Treating as a root run.


AIMessage(content='이전의 답변은 번역되었습니다. 필요에 따라 해당 답변의 번역을 제공합니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-06-21T16:08:54.006842287Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 509897221, 'load_duration': 2826514, 'prompt_eval_count': 32, 'prompt_eval_duration': 102865000, 'eval_count': 27, 'eval_duration': 271853000}, id='run-e2afc4d9-b88d-4e75-a71f-bfe6267d7e8a-0')